Axioms for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.physics.quantum.QPE</a>
========

In [1]:
import proveit
# Prepare this notebook for defining the axioms of a theory:
%axioms_notebook # Keep this at the top following 'import proveit'.
from proveit import Literal
from proveit import a, b, eps, k, l, n, t
from proveit.linalg import MatrixProd, ScalarProd, TensorExp 
from proveit.logic import Equals, Forall, InSet
from proveit.numbers import zero, one, two, e, i, pi, Div, Exp, Integer, Interval, IntervalCO, NaturalPos 
from proveit.numbers import Add, Exp, Floor, frac, LessEq, Mod, ModAbs, Mult, Prod, subtract, Sum
from proveit.physics.quantum import Ket, Meas, RegisterBra, RegisterSU, QubitRegisterSpace
from proveit.physics.quantum.QPE import (alpha_l, b_, delta_, m_, n_, phase_,
                                                  phase_m_, psi_k, Psi_, t_, two_pow_t,
                                                  u_, U_ )
from proveit.physics.quantum.QPE.phase_est_ops import ModAdd, Pfail, Psuccess
from proveit.statistics import Prob

In [2]:
%begin axioms

Defining axioms for theory 'proveit.physics.quantum.QPE'
Subsequent end-of-cell assignments will define axioms
%end_axioms will finalize the definitions


Let $U$ be a unitary operator that acts on $n$ qubits, with $\lvert u\rangle$ as an eigenstate of $U$ with eigenvalue $e^{2 \pi i \varphi}$:

In [3]:
n_in_nat_pos = InSet(n_, NaturalPos)

n_in_nat_pos: n in NaturalPos

In [4]:
unitary_u = InSet(U_, RegisterSU(n_))

unitary_u: U in [SU](2^{n})

In [5]:
u_ket_register = InSet(Ket(u_), QubitRegisterSpace(n_))

u_ket_register: |u〉 in (Complex^{2})^{otimes n}

In [6]:
phase_in_interval = InSet(phase_, IntervalCO(zero, one))

phase_in_interval: phase in [0,1)

In [7]:
eigen_uu = Equals(MatrixProd(U_, Ket(u_)), ScalarProd(Exp(e, Mult(two, pi, i, phase_)), Ket(u_)))

eigen_uu: (U . |u〉) = (e^{2 * pi * i * phase} * |u〉)

In [8]:
# t (represented by the Literal t_)
# denotes the number of Qbits in the input register
t_in_natural_pos = InSet(t_, NaturalPos)

t_in_natural_pos: t in NaturalPos

In [9]:
psi_kets_in_q_reg_space = Forall(k, InSet(Ket(psi_k), QubitRegisterSpace(k)), domain=NaturalPos)

psi_kets_in_q_reg_space: forall_{k in NaturalPos} (|psi_{k}〉 in (Complex^{2})^{otimes k})

In [10]:
PsiKetInQRegSpace = InSet(Ket(Psi_), QubitRegisterSpace(t_))

PsiKetInQRegSpace: |Psi〉 in (Complex^{2})^{otimes t}

In [11]:
# first_q_p_e = Circuit([[Input(ket0), Hgate, CTRL_DN, Output(Ket(psi_1))],
#                 [Input(Ket(u_)), MultiWire(n_), Gate(U_), Output(Ket(u_))]])

In [12]:
# mid_q_p_e = Forall(k, 
#                Circuit([[Input(Ket(psi_k)), MultiWire(k), PASS, WIRE_DN, PASS, PASS],
#                 [Input(ket0), Hgate, CTRL_DN, WIRE_LINK, MultiWire(Add(k, one)), Output(Ket(psi_next))],
#                 [Input(Ket(u_)), MultiWire(n_), Gate(U_pow_two_pow_k), Output(Ket(u_)), PASS, PASS]]),
#        domain=NaturalPos)

In [13]:
# final_q_p_e = Circuit([[Input(Ket(psi_t)), MultiWire(t_), Gate(InverseFourierTransform(t_)), Output(Ket(Psi_))]])

In [14]:
# trying here to define Psi_1 (the result at the end of "stage 1")
# represented in Nielsen & Chuang's Ex 5.20 (pg 222)
Psi_1 = Literal(string_format='Psi_1', latex_format=r'\Psi_{1}')
phi_ = Literal(string_format='phi', latex_format=r'\varphi')
Psi_1_def = Equals(Psi_1,
                   Mult(frac(one, Exp(two, frac(t_,two))),
                        Sum(k, Mult(Exp(e, Mult(two, pi, i, phase_, k)), Ket(k)), domain=Interval(zero, subtract(Exp(two, t_), one)))))

Psi_1_def: Psi_1 = ((1 / 2^{t / 2}) * (Sum_{k=0}^{2^{t} - 1} (e^{2 * pi * i * phase * k} * |k〉)))

Let $\lvert \Psi \rangle$ be the outcome of the $t$-qubit register of ${\rm QPE}(U, t)$ acting on $\lvert u \rangle$, $m$ be a random variable representing the measurement of Psi with the register interpretted as an integer (via binary representation), and $\varphi_m = 2 \pi m/2^t$ be the random variable phase outcome of ${\rm QPE}(U, t)$:

In [15]:
m_def = Equals(m_, Meas(Ket(Psi_)))


m_def: m = MEAS(|Psi〉)

In [16]:
phase_m_def = Equals(phase_m_, frac(m_, two_pow_t))

phase_m_def: phase_m = (m / 2^{t})

Let $b$ be the value for $m$ that gives the closest $\varphi_m$ to $\varphi$ without exceeding it, and let $\delta$ be this difference:

In [17]:
best_def = Equals(b_, Floor(Mult(phase_, two_pow_t)))

best_def: b = floor(phase * 2^{t})

In [18]:
delta_def = Equals(delta_, subtract(phase_, frac(b_, two_pow_t)))

delta_def: delta = (phase - (b / 2^{t}))

The probability of success is defined as $\theta_m$ being within some epsilon of $\theta$: 

In [19]:
success_def = Forall(
        eps,
        Equals(Psuccess(eps),
               Prob(LessEq(ModAbs(subtract(m_, b_), two_pow_t), eps), m_)), 
        domain=NaturalPos)

success_def: forall_{eps in NaturalPos} ([Psuccess](eps) = Pr_{m}[|m - b|_{mod 2^{t}} <= eps])

In [20]:
fail_def = Forall(
    eps,
    Equals(Pfail(eps), subtract(one, Psuccess(eps))),
    domain=NaturalPos)

fail_def: forall_{eps in NaturalPos} ([Pfail](eps) = (1 - [Psuccess](eps)))

Let $\alpha_l$ be the amplitude of $\lvert \Psi \rangle$ for the $b \oplus l$ state, where $\oplus$ is defined as addition modulo $2^t$:

In [21]:
mod_add_def = Forall(
    (a, b),
    Equals(ModAdd(a, b), Mod(Add(a, b), two_pow_t)),
    domain=Integer)

mod_add_def: forall_{a, b in Integer} ((a MOD_ADD b) = ((a + b) mod  2^{t}))

In [22]:
alpha_def = Forall(
    l,
    Equals(alpha_l,
           MatrixProd(RegisterBra(ModAdd(b_, l), t_),
                      Ket(Psi_))),
    domain=Integer)

alpha_def: forall_{l in Integer} (alpha_{l} = {t}_〈b MOD_ADD l|Psi〉)

In [23]:
%end axioms

These axioms may now be imported from the theory package: proveit.physics.quantum.QPE
